In [ ]:
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
from utils import phase_from_frequency, sample_space, sample_rate

## Altering the Sounds

Now that we know how to create sounds, lets look at the ways we can alter or modify these sounds

#### Modulators
Modulation means altering certain characteristics of a sound wave, typically the amplitude, frequency, or phase of an audio signal, using a modulating source. This modulating source, often referred to as a modulator, affects the parameters of a carrier waveform (usually another audio signal) to create more complex and rich sound textures. We also have a modulation index which is a measure of extent of modulation done on a carrier signal

#### FM
In frequency modulation, the frequency of a signal is changed to encode information. The receiver decodes this information. We widely use this in FM radio broadcasting.

let's say you and I have a string where each of us holds one end. If I pull on it in a constant pattern, say one tug per second, that would be our frequency. Now in order to send information we agree of a certain code (i.e A is two tugs in a second, B is three, C is four, etc.) Now to send you a message via string, I tug the appropriate number of times per second, varying the frequency of my tugs to match our code. This is frequency modulation in simple terms.


![text](https://cdn.prod.website-files.com/620d42e86cb8ec4d0839e59d/6230ec578c0e62f4b85ee667_61ca13728a684337b0f7598f_Waveform-for-frequency-modulation.jpeg)

Why FM is Used for Radio Broadcasting

- High Fidelity: Better audio quality than AM radio
- Noise Resistance: Less affected by atmospheric and electrical interference
- Stereo Capability: Supports stereo transmission while maintaining mono compatibility


```
FM signal: A_c * sin(ω_c * t + β * sin(ω_m * t))
```


In [ ]:
# Carrier signal
carrier_freq = 400  # Hz
carrier_amplitude = 1.0
carrier_omega = phase_from_frequency(carrier_freq)
carrier_signal = np.sin(carrier_omega)

# Modulating signal
modulating_freq = 50  # Hz
modulating_omega = phase_from_frequency(modulating_freq)
modulating_signal = np.sin(modulating_omega)

modulation_index = 5  # Modulation depth/sensitivity (Controls the extent of frequency deviation)

# Generate FM signal: sin(ω_c * t + β * sin(ω_m * t))
fm_signal = carrier_amplitude * np.sin(carrier_omega + modulation_index * modulating_signal)

In [ ]:
ipd.Audio(fm_signal, rate=sample_rate)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1)
plt.plot(sample_space[:10000], carrier_signal[:10000])
plt.title('Carrier Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 2)
plt.plot(sample_space[:10000], modulating_signal[:10000])
plt.title('Modulating Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 3)
plt.plot(sample_space[:10000], fm_signal[:10000])
plt.title('Modulated Signal (Frequency Modulation)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

### [E] AM
In amplitude modulation, the amplitude of a carrier waveform is modulated by a separate modulating waveform. The modulating waveform affects the volume or amplitude of the carrier waveform, creating harmonic sidebands around the carrier frequency. When applied in a synthesizer, AM can produce metallic or bell-like sounds, and it's useful for creating a variety of tonal textures.

```
AM signal: A_c * (1 + β * sin(ω_m * t)) * sin(ω_c * t)
```

AM (Amplitude Modulation) is used when long-distance transmission is the priority and cost is a major factor. FM (Frequency Modulation) is used when superior audio quality and noise immunity are needed, even though it has a shorter transmission range and is more expensive

In [ ]:
# Generate AM signal: A_c * (1 + μ * sin(ω_m * t)) * sin(ω_c * t)
# where μ is the modulation depth (0-1)
am_signal = carrier_amplitude * (1 + modulating_signal) * np.sin(carrier_omega)

In [ ]:
# ipd.Audio(am_signal, rate=sample_rate)

## Music and Modulation
Modulation can be used to create very unique sounds and can allow changing of timbre and these harmonics would change depending on the kind of modulation and the signal used, In the end we can produce create more intricate and diverse sounds

#### FM

FM can be used with non-harmonic ratios between carrier and modulator frequencies to create characteristic bell-like sounds. Higher modulation indices produce more inharmonic overtones

In [ ]:
def create_fm_tone(carrier_freq, mod_freq, mod_index, amplitude=0.5, t=sample_space):
    carrier_omega = phase_from_frequency(carrier_freq)
    mod_omega = phase_from_frequency(mod_freq)
    
    # Pure FM synthesis: carrier + modulator
    modulator = np.sin(mod_omega)
    fm_signal = amplitude * np.sin(carrier_omega * t + mod_index * modulator)
    
    # Normalize
    fm_signal = fm_signal / np.max(np.abs(fm_signal))
    return fm_signal

In [ ]:
# Bell-like sound (C:M ratio near 1:1.4)
bell_tone = create_fm_tone(
    carrier_freq=200,
    mod_freq=280,
    mod_index=5.0
)
ipd.Audio(bell_tone, rate=sample_rate)

In [ ]:
# Metallic Chime (non-integer C:M ratio around 1:3.5)
metal_hit = create_fm_tone(
    carrier_freq=350,
    mod_freq=1470,
    mod_index=10.0
)
ipd.Audio(metal_hit, rate=sample_rate)

**[E] Try out different Ratios of Carrier_Freq / Mod_Freq / Mod_Index**

### Tremolo Effect

This is the most basic AM effect, where amplitude varies periodically, creating a pulsating sound. It is made by variation in amplitude of sound and is also called underwater effect. Tremolo uses the AM equation: `output = carrier * (1 + modulator)` where the modulator is a low-frequency oscillator (LFO) typically between 1-20 Hz.

Tremolo is one of the oldest electronic effects in music, used extensively in guitar amplifiers, organs, and synthesizers. Tremolo maintains the original pitch and simply varies the amplitude, creating a characteristically pulsating, rhythmic quality to sustained notes.

In [ ]:
audio_url = "static/tremolo.mp3"
ipd.Audio(audio_url)

In [ ]:
def create_tremolo(carrier_signal, tremolo_rate, tremolo_depth):
    # Create modulator signal (LFO)
    modulator = tremolo_depth * np.sin(phase_from_frequency(tremolo_rate))
    
    # Apply AM modulation: carrier * (1 + modulator)
    # We normalize the modulator to ensure it stays in the range [-depth, depth]
    tremolo_signal = carrier_signal * (1 + modulator)
    
    # Normalize output
    tremolo_signal = tremolo_signal / np.max(np.abs(tremolo_signal))
    
    return tremolo_signal

### Tremolo Parameters

- **Rate:** Also known as Speed, It is how quickly the volume fluctuates
- **Depth:** How much the volume changes, Its the "intensity" or "strength" of the tremolo.

A slower **Rate** and higher **Depth** create a dramatic pulsing, whereas a faster Rate and lower Depth results in a subtle, shimmering effect

<img src="https://m.media-amazon.com/images/I/61EjofqR3kL._AC_UF1000,1000_QL80_.jpg" alt="alt" width="300" />

In [ ]:
carrier_freq = 440  # A4 note
carrier_amplitude = 0.8
carrier_signal = carrier_amplitude * np.sin(phase_from_frequency(carrier_freq))

# Play around with the Rate and Depth #

tremolo_rate = 4  # 4, 6.5, 16 (different rates)
tremolo_depth = 0.5  # 0.5, 2, 5 Depth
tremolo = create_tremolo(carrier_signal, tremolo_rate, tremolo_depth)
ipd.Audio(tremolo, rate=sample_rate)

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(sample_space[:65000], tremolo[:65000])